## Output Passer

Quando você usa um modelo de linguagem, ele geralmente retorna uma string de texto. Mas muitas vezes, você quer que essa resposta seja:

Um objeto Python (como um dicionário ou lista)
Um valor específico (como um número ou booleano)
Um JSON válido
Ou até mesmo uma estrutura personalizada (como uma classe ou schema)
É aí que entra o Output Parser: ele converte a saída textual do modelo nesse formato desejado.

In [ ]:
feedbackProdutp="""
O XPhone Z10 se destaca pelo seu design sofisticado, desempenho potente e excelente qualidade de tela e câmera. A bateria também impressiona pela longa duração. No entanto, o aparelho apresenta alguns pontos negativos, como superaquecimento durante jogos, falhas no reconhecimento facial em ambientes escuros, lentidão nas atualizações de sistema e qualidade de som abaixo do esperado. Para melhorar a experiência do usuário, recomenda-se otimizações no sistema de resfriamento, melhorias no reconhecimento facial, aceleração das atualizações e inclusão de alto-falantes estéreo nas próximas versões.
"""



In [ ]:
from langchain.prompts import ChatPromptTemplate

reviewTemplate = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:
- produto: Nome do produto mencionado no texto.
- caracteristicas_positivas: Liste todas as características positivas mencionadas.
- caracteristicas_negativas: Liste todas as características negativas mencionadas.
- recomendacoes: Sugestões de melhoria ou solução para os problemas identificad.
texto{review}

retone a resposta no formato json
""")


In [ ]:
print(reviewTemplate.format_messages(review=feedbackProdutp))

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

schemaProduto= ResponseSchema(
    name="produto",
    type="string",
    description="nome do produto"
)
schemaPositiva= ResponseSchema(
    name="caracteristicas possitiva",
    type="list",
    description="Liste todas as características positivas mencionadas"
)
schemaNegativas= ResponseSchema(
    name="caracteristicas negativas",
    type="list",
    description="Liste todas as características negativas mencionadas"
)
schemaRecomendacao= ResponseSchema(
    name="recomendação",
    type="list",
    description="Sugestões de melhoria ou solução para os problemas identificad"
)

In [ ]:
response_schema = [schemaProduto,schemaPositiva,schemaNegativas,schemaRecomendacao]
output_parser= StructuredOutputParser.from_response_schemas(response_schema)

schemaFormatado= output_parser.get_format_instructions()

In [ ]:
print(schemaFormatado)

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Corrigindo o nome do parâmetro 'schama' para 'schema'
reviewTemplate2 = ChatPromptTemplate.from_template("""
    Para o texto a seguir, extraia as seguintes informações:
    produtos, características positivas, características negativas e recomendação.
    
    Texto: {review}
    {schema}
""", partial_variables={"schema": schemaFormatado})

# Formatando a mensagem com o texto da review
mensagem_formatada = reviewTemplate2.format_messages(review=feedbackProdutp)

# Exibindo a mensagem formatada
print(mensagem_formatada)


In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

resposta= chat.invoke(reviewTemplate2.format_messages(review=feedbackProdutp))
resposta.content

In [ ]:
respostaJson=output_parser.parse(resposta.content)
